In [16]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns

In [17]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector']]

# Function to filter the S&P 500 companies by sector
def filter_energy_sector(df):
    return df[df['GICS Sector'] == 'Energy']['Symbol'].tolist()

# Get the list of S&P 500 companies and their sectors
sp500_companies = get_sp500_companies()

# Filter the S&P 500 companies to include only the ones in the energy sector
energy_stocks = filter_energy_sector(sp500_companies)

# Print the list of S&P 500 energy sector stocks
print(energy_stocks)


['APA', 'BKR', 'CVX', 'COP', 'CTRA', 'DVN', 'FANG', 'EOG', 'EQT', 'XOM', 'HAL', 'HES', 'KMI', 'MRO', 'MPC', 'OXY', 'OKE', 'PSX', 'SLB', 'TRGP', 'VLO', 'WMB']


In [25]:
# Define the ticker symbols as a list
ticker_symbols = ['SYK']
window_days = 90
# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)

# Create an empty DataFrame to store the adjusted data
adjusted_stock_data = pd.DataFrame()
earnings_data = {}  # Dictionary to store earnings dates for each ticker
metric_list = []

# Fetch historical data for each ticker symbol and adjust the format
for symbol in ticker_symbols:

    # symbol's historical price related
    data = yf.download(symbol, start=start_date, end=end_date)
    data = data.sort_index(ascending=False)
    data.index = [date.date() for date in data.index.sort_values(ascending=False)] # convert original timestamp index to date index

    # symbol's earning related
    stock = yf.Ticker(symbol)
    earnings_dates = stock.get_earnings_dates(limit=12)  # Fetch earnings dates for the past 4 quarters and forecast for the next 4 quarters
    earnings_data[symbol] = earnings_dates


    # Extract close prices and add them to the adjusted DataFrame with ticker symbol as column name
    adjusted_stock_data[symbol] = data['Close'].round(2)
    adjusted_stock_data[symbol + "_EPS_TTM"] = None


    # EPS TTM calculation
    EPS_TTM_start_date = data.index
    for i in EPS_TTM_start_date:
        for key, value in earnings_data.items():
            if key == symbol:
                symbol_hist_EPS = value['Reported EPS'][value['Reported EPS'].notnull()]
                symbol_hist_EPS.index = [date.date() for date in symbol_hist_EPS.index.sort_values(ascending=False)] # convert timestamp index to a date index(sorted, descending)

                symbol_projected_EPS = value['EPS Estimate'][value['EPS Estimate'].notnull()]
                symbol_projected_EPS.index = [date.date() for date in symbol_projected_EPS.index.sort_values(ascending=False)] # convert timestamp index to a date index(sorted, descending)

                # Filter the DataFrame to include only dates(index) less than or equal to the target date
                filtered_symbol_hist_EPS = symbol_hist_EPS[symbol_hist_EPS.index <= i]
                
                # Select the first four rows from the past_4_qtrs_EPS
                past_4_qtrs_EPS = filtered_symbol_hist_EPS.head(4)      

                # Select the first rows of estimate to be the latest qtrly projected EPS
                latest_projected_EPS = symbol_projected_EPS.head(1) 
                
                # # Calculate the sum of the numeric values in the selected rows
                EPS_TTM = past_4_qtrs_EPS.sum() 
                EPS_latest_projected = latest_projected_EPS.sum() + filtered_symbol_hist_EPS.head(3).sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS


                adjusted_stock_data.loc[i, f"{symbol}_EPS_TTM"] = EPS_TTM
                adjusted_stock_data[f"{symbol}_EPS_latest_projected"] = EPS_latest_projected

            else:
                continue
            

    # stock's stats

    adjusted_stock_data[f"{symbol}_PE_TTM"] = adjusted_stock_data[symbol] / adjusted_stock_data[f"{symbol}_EPS_TTM"]
    adjusted_stock_data[f"{symbol}_PE_TTM_avg"] = adjusted_stock_data[f"{symbol}_PE_TTM"].mean().round(2)
    adjusted_stock_data[f"{symbol}_PE_TTM_std"] = np.std(adjusted_stock_data[f"{symbol}_PE_TTM"]).round(2)
    adjusted_stock_data[f"{symbol}_PE_TTM_volatility_+"] = adjusted_stock_data[f"{symbol}_PE_TTM_avg"] + adjusted_stock_data[f"{symbol}_PE_TTM_std"] # 这个是PE的波动范围上限
    adjusted_stock_data[f"{symbol}_PE_TTM_volatility_-"] = adjusted_stock_data[f"{symbol}_PE_TTM_avg"] - adjusted_stock_data[f"{symbol}_PE_TTM_std"] # 这个是PE的波动范围下限

    adjusted_stock_data[f"{symbol}_relative_valuation_+"] = adjusted_stock_data[f"{symbol}_PE_TTM_volatility_+"] * adjusted_stock_data[f"{symbol}_EPS_TTM"] # 这个是relative valuation的价格上限
    adjusted_stock_data[f"{symbol}_relative_valuation_-"] = adjusted_stock_data[f"{symbol}_PE_TTM_volatility_-"] * adjusted_stock_data[f"{symbol}_EPS_TTM"] # 这个是relative valuation的价格下限
    adjusted_stock_data[f"{symbol}_relative_valuation_median"] = np.median([adjusted_stock_data[f"{symbol}_relative_valuation_+"].head(1), adjusted_stock_data[f"{symbol}_relative_valuation_-"].head(1)]) #这个是根据最新TTM PE估值的价格中位数

    adjusted_stock_data[f"{symbol}_relative_valuation_projected_+"] = adjusted_stock_data[f"{symbol}_PE_TTM_volatility_+"] * adjusted_stock_data[f"{symbol}_EPS_latest_projected"] # 这个是relative valuation的价格上限
    adjusted_stock_data[f"{symbol}_relative_valuation_projected_-"] = adjusted_stock_data[f"{symbol}_PE_TTM_volatility_-"] * adjusted_stock_data[f"{symbol}_EPS_latest_projected"] # 这个是relative valuation的价格下限
    adjusted_stock_data[f"{symbol}_relative_valuation_projected_median"] = np.median([adjusted_stock_data[f"{symbol}_relative_valuation_projected_+"], adjusted_stock_data[f"{symbol}_relative_valuation_projected_-"]]) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    adjusted_stock_data[f"{symbol}_{window_days}_price_min"] = adjusted_stock_data[symbol].min().round(2)
    adjusted_stock_data[f"{symbol}_{window_days}_price_max"] = adjusted_stock_data[symbol].max().round(2)
    adjusted_stock_data[f"{symbol}_{window_days}_price_avg"] = adjusted_stock_data[symbol].mean().round(2)
    adjusted_stock_data[f"{symbol}_{window_days}_price_std"] = np.std(adjusted_stock_data[symbol]).round(2)



    conditions = [
    (adjusted_stock_data[f"{symbol}"] < adjusted_stock_data[f"{symbol}_relative_valuation_-"]),
    (adjusted_stock_data[f"{symbol}"] > adjusted_stock_data[f"{symbol}_relative_valuation_+"]),
    ((adjusted_stock_data[f"{symbol}"] >= adjusted_stock_data[f"{symbol}_relative_valuation_-"]) & (adjusted_stock_data[f"{symbol}"] <= adjusted_stock_data[f"{symbol}_relative_valuation_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    adjusted_stock_data[f"{symbol}_curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        adjusted_stock_data.loc[condition, f"{symbol}_price_valuation_assessment"] = category




# # Reset index to make it cleaner
# adjusted_stock_data.reset_index(inplace=True)

# # Example usage: Display the first few rows of the adjusted DataFrame
# adjusted_stock_data.sort_values(by='Date', ascending=False)

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
selected_cols = [
    f"{symbol}"
    ,f"{symbol}_PE_TTM"
    ,f"{symbol}_PE_TTM_avg"
    ,f"{symbol}_relative_valuation_+"
    ,f"{symbol}_relative_valuation_-"
    ,f"{symbol}_relative_valuation_median"
    ,f"{symbol}_relative_valuation_projected_+"
    ,f"{symbol}_relative_valuation_projected_-"
    ,f"{symbol}_relative_valuation_projected_median"
    ,f"{symbol}_price_valuation_assessment"
]

# adjusted_stock_data.columns
adjusted_stock_data[selected_cols]

[*********************100%%**********************]  1 of 1 completed


,SYK,SYK_PE_TTM,SYK_PE_TTM_avg,SYK_relative_valuation_+,SYK_relative_valuation_-,SYK_relative_valuation_median,SYK_relative_valuation_projected_+,SYK_relative_valuation_projected_-,SYK_relative_valuation_projected_median,SYK_price_valuation_assessment
2024-05-09,330.86,27.756711,29.73,375.48,333.2832,354.3816,354.06,314.2704,334.1652,undervalued
2024-05-08,329.28,27.717172,29.73,374.22,332.1648,354.3816,354.06,314.2704,334.1652,undervalued
2024-05-07,332.14,27.957912,29.73,374.22,332.1648,354.3816,354.06,314.2704,334.1652,undervalued
2024-05-06,325.50,27.39899,29.73,374.22,332.1648,354.3816,354.06,314.2704,334.1652,undervalued
2024-05-03,328.45,27.647306,29.73,374.22,332.1648,354.3816,354.06,314.2704,334.1652,undervalued
2024-05-02,326.63,27.494108,29.73,374.22,332.1648,354.3816,354.06,314.2704,334.1652,undervalued
2024-05-01,328.12,27.619529,29.73,374.22,332.1648,354.3816,354.06,314.2704,334.1652,undervalued
2024-04-30,336.50,28.324916,29.73,374.22,332.1648,354.3816,354.06,314.2704,334.1652,fair
2024-04-29,338.87,28.428691,29.73,375.48,333.2832,354.3816,354.06,314.2704,334.1652,fair
2024-04-26,335.61,28.155201,29.73,375.48,333.2832,354.3816,354.06,314.2704,334.1652,fair


In [22]:
def get_earnings_dates(ticker_symbols):
    earnings_data = {}  # Dictionary to store earnings dates for each ticker
    
    for ticker in ticker_symbols:
        stock = yf.Ticker(ticker)
        earnings_dates = stock.get_earnings_dates(
            limit=12
        )  # Fetch earnings dates for the past 4 quarters and forecast for the next 4 quarters
        earnings_data[ticker] = earnings_dates
        
    return earnings_data


earnings_data = get_earnings_dates(ticker_symbols)


for ticker, dates in earnings_data.items():
    if ticker in ['ACN']:
        print(f"Earnings Dates for {ticker}:")
        print(dates)
        print()
    else:
        continue

Earnings Dates for ACN:
                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-01-30 07:00:00-05:00           NaN           NaN          NaN
2024-09-26 08:00:00-04:00           NaN           NaN          NaN
2024-06-20 08:00:00-04:00          3.15           NaN          NaN
2024-06-20 08:00:00-04:00          3.15           NaN          NaN
2024-03-21 06:00:00-04:00          2.66          2.77       0.0402
2024-01-31 07:00:00-05:00          3.14          3.27       0.0414
2023-12-19 06:00:00-05:00          3.14          3.27       0.0401
2023-09-28 06:00:00-04:00          2.66          2.71       0.0200
2023-06-22 06:00:00-04:00          3.04          3.19       0.0500
2023-03-23 06:00:00-04:00          2.50          2.69       0.0769
2022-12-16 06:00:00-05:00          2.91          3.08       0.0577
2022-09-22 06:00:00-04:00          2.57          2.60       0.0114



In [23]:
earnings_data = get_earnings_dates(ticker_symbols)


for ticker, dates in earnings_data.items():
    if ticker == 'ACN':
        print(f"Earnings Dates for {ticker}:")
        print(dates)
        print()

Earnings Dates for ACN:
                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-01-30 07:00:00-05:00           NaN           NaN          NaN
2024-09-26 08:00:00-04:00           NaN           NaN          NaN
2024-06-20 08:00:00-04:00          3.15           NaN          NaN
2024-06-20 08:00:00-04:00          3.15           NaN          NaN
2024-03-21 06:00:00-04:00          2.66          2.77       0.0402
2024-01-31 07:00:00-05:00          3.14          3.27       0.0414
2023-12-19 06:00:00-05:00          3.14          3.27       0.0401
2023-09-28 06:00:00-04:00          2.66          2.71       0.0200
2023-06-22 06:00:00-04:00          3.04          3.19       0.0500
2023-03-23 06:00:00-04:00          2.50          2.69       0.0769
2022-12-16 06:00:00-05:00          2.91          3.08       0.0577
2022-09-22 06:00:00-04:00          2.57          2.60       0.0114

